<a href="https://colab.research.google.com/github/HatemMoushir/Sentiment/blob/main/ArEn-TweetSentiment-BERT-Hatem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ تثبيت المكتبات المطلوبة
!pip install -q datasets transformers evaluate pandas scikit-learn

import re
import pandas as pd
import numpy as np
from datasets import load_dataset, concatenate_datasets, Dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate
import os

file_path_ar_dataset = "/content/Arabic Tweets Sentiment Classification 2024/Arabic_tweets_sentiment.csv"
file_path_en_dataset = "/content/training.1600000.processed.noemoticon.csv"

if os.path.exists(file_path_ar_dataset):
     print("✅ الملف موجود.")
     print(file_path_ar_dataset)
     print("-----")else:
     print("❌ الملف غير موجود.")
     !wget -q https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/m88gg52wp7-1.zip -O arabicdata1.zip
     !unzip -o arabicdata1.zip


# ✅ تحميل بيانات التغريدات العربية من UCI



# Read the CSV with the correct separator and rename the column
#ds_ar = pd.read_csv("/content/Arabic Tweets Sentiment Classification 2024/Arabic_tweets_sentiment.csv", encoding='utf-8', sep='\t')
ds_ar = pd.read_csv(file_path_ar_dataset, encoding='utf-8', sep='\t')

ds_ar.rename(columns={'class': 'class'}, inplace=True)


# فلترة السجلات التي تحتوي فقط على Positive أو Negative
ds_ar = ds_ar[ds_ar["class"].isin(["Positive", "Negative"])]


# 🧼 تنظيف النصوص الإنجليزية
def clean_english_text(text):
    text = re.sub(r"http\S+", "", text)     # remove URLs
    text = re.sub(r"@\w+", "", text)        # remove mentions
    text = re.sub(r"#", "", text)           # remove hashtags
    text = re.sub(r"[^\w\s]", "", text)     # remove punctuation
    text = re.sub(r"\d+", "", text)         # remove numbers
    return text.lower().strip()

# 🧼 تطبيع وتنظيف النصوص العربية
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def clean_arabic_text(text):
    text = normalize_arabic(text)
    text = re.sub(r"http\S+", "", text)  # remove URLs
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)  # remove non-Arabic
    text = re.sub(r"\s+", " ", text)     # normalize whitespace
    return text.strip()

# 🗂️ تجهيز بيانات التغريدات العربية من UCI

# تنظيف وتطبيع
ds_ar["text"] = ds_ar["text"].fillna("").apply(clean_arabic_text)

ds_ar["label"] =ds_ar["class"].replace({"Negative": 0, "Positive": 1}).astype(int)


# اختيار 1000 عينة
ds_ar = ds_ar.sample(50000, random_state=42).reset_index(drop=True)

# تحويل إلى Dataset
ds_ar = Dataset.from_pandas(ds_ar[["text", "label"]])

# 2️⃣ تحميل 1000 تغريدة إنجليزية من Sentiment140 (Stanford)

if os.path.exists(file_path_en_dataset):
     print("✅ الملف موجود.")
     print(file_path_en_dataset)
     print("-----")
else:
      print("❌ الملف غير موجود.")
      !wget -q https://cs.stanford.edu/people/alecmgo/tra.iningandtestdata.zip -O sentiment140.zip
      !unzip -o sentiment140.zip

import pandas as pd
import os

cols = ['label', 'id', 'date', 'query', 'user', 'text']
#ds_en = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', names=cols)

ds_en = pd.read_csv(file_path_en_dataset, encoding='latin-1', names=cols)

# 3. تحويل التصنيفات:
# 0 → سلبي | 4 → إيجابي → نحولها لـ 1
ds_en = ds_en[ds_en['label'].isin([0, 4])]
ds_en['label'] =ds_en['label'].map({0: 0, 4: 1})

# 4. تقليل الحجم لعينة صغيرة للتجربة
ds_en = ds_en.sample(50000, random_state=42).reset_index(drop=True)

# 5. تحويل إلى Dataset
from datasets import Dataset
ds_en = Dataset.from_pandas(ds_en[['text', 'label']])

# 3️⃣ دمج اللغتين في Dataset واحد
ds = concatenate_datasets([
    ds_ar,
    ds_en
]).shuffle(seed=44)

print("✅ إجمالي العبارات:", len(ds))

# 4️⃣ إعداد Tokenizer و Model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# 5️⃣ تشفير النصوص
def tok(x):
    return tokenizer(x["text"], truncation=True, padding="max_length", max_length=128)

ds = ds.map(tok, batched=True)

# 6️⃣ تقسيم البيانات
split = ds.train_test_split(test_size=0.1, seed=45)
train_ds, test_ds = split["train"], split["test"]

# 7️⃣ إعداد التدريب
accuracy = evaluate.load("accuracy")
os.environ["WANDB_DISABLED"] = "true"

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=-1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds)
    }

training_args = TrainingArguments(
    output_dir="sentiment_ar_en_model",
    eval_strategy="epoch", # Changed back to evaluation_strategy
    #evaluation_steps=50, # Added evaluation_steps
    save_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    logging_steps=50,
    load_best_model_at_end=True,
    report_to=None
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 8️⃣ بدء التدريب
trainer.train()

# 9️⃣ حفظ النموذج
trainer.save_model("sentiment_model_final")
tokenizer.save_pretrained("sentiment_model_final")

In [ ]:
from huggingface_hub import upload_folder

hf_username = "HatemMoushir"
repo_name = "ArEn-TweetSentiment-BERT-Hatem"


upload_folder(
    folder_path="/content/sentiment_model_final",

    repo_id=f"{hf_username}/{repo_name}",
    repo_type="model"
)

In [ ]:


from transformers import pipeline

# تحميل النموذج
classifier = pipeline("sentiment-analysis", model="HatemMoushir/ArEn-TweetSentiment-BERT-Hatem")

# 100 جملة مع التصنيفات الحقيقية (1 = إيجابي، 0 = سلبي)
samples = [
    ("أنا سعيد جدًا اليوم", 1),
    ("الجو ممطر وهذا يجعلني حزينًا", 0),
    ("نجحت في الامتحان!", 1),
    ("أشعر بالإحباط من الأخبار", 0),
    ("أحب أصدقائي كثيرًا", 1),
    ("هذا أسوأ يوم في حياتي", 0),
    ("أشعر بالراحة والطمأنينة", 1),
    ("لم أتمكن من النوم جيدًا الليلة", 0),
    ("اليوم جميل ومشمس", 1),
    ("كل شيء يسير بشكل خاطئ", 0),
    ("أحب مشاهدة الأفلام مع عائلتي", 1),
    ("تأخرت عن العمل وفقدت مزاجي", 0),
    ("أشعر بالنشاط والحيوية", 1),
    ("المكان مزدحم ولا أستطيع التحمل", 0),
    ("قضيت عطلة رائعة على الشاطئ", 1),
    ("انتهى اليوم بشكل سيء", 0),
    ("أشعر بالتفاؤل بشأن المستقبل", 1),
    ("لم يعجبني الطعام اليوم", 0),
    ("أشعر بالحب من الجميع", 1),
    ("خسرت كل شيء في لحظة", 0),
    ("الموسيقى تجعلني سعيدًا", 1),
    ("الطريق مزدحم وأنا غاضب", 0),
    ("أنا ممتن لكل شيء لدي", 1),
    ("كان يومًا مرهقًا جدًا", 0),
    ("أشعر بالأمل رغم الصعوبات", 1),
    ("لا أطيق الانتظار لزيارة أصدقائي", 1),
    ("تجاهلني في الاجتماع وشعرت بالإهانة", 0),
    ("فزت في المسابقة!", 1),
    ("الجو خانق ولا يُحتمل", 0),
    ("تلقيت رسالة جميلة من صديقي", 1),
    ("انقطعت الكهرباء وفاتني الفيلم", 0),
    ("أنا محظوظ بعائلتي", 1),
    ("لا أحد يهتم بي", 0),
    ("الهدوء في هذا المكان يريحني", 1),
    ("خسرت فرصتي الأخيرة", 0),
    ("أشعر أنني محبوب", 1),
    ("ضاعت أمتعتي في المطار", 0),
    ("قمت بعمل جيد اليوم", 1),
    ("لا أريد التحدث مع أحد", 0),
    ("أنا ممتن للحياة", 1),
    ("يوم ممل وبلا فائدة", 0),
    ("تلقيت ترقية في العمل", 1),
    ("أشعر بالإجهاد والتعب", 0),
    ("الهدية أسعدتني كثيرًا", 1),
    ("انهرت من الضغط", 0),
    ("تناولت وجبة لذيذة", 1),
    ("تأخرت الرحلة وأشعر بالضيق", 0),
    ("حققت هدفًا كنت أسعى له", 1),
    ("الخسارة كانت قاسية", 0),
    ("أنا فخور بنفسي", 1),
    ("فقدت الثقة في من حولي", 0),
    ("عطلة نهاية الأسبوع كانت رائعة", 1),
    ("لا أجد أي دافع للاستمرار", 0),
    ("ابني نجح في دراسته", 1),
    ("كل من حولي خذلني", 0),
    ("مشيت على البحر وكان الجو جميلًا", 1),
    ("تعرضت لموقف محرج أمام الجميع", 0),
    ("أشعر بالسعادة لأني ساعدت شخصًا", 1),
    ("تم تجاهلي بالكامل", 0),
    ("نمت جيدًا واستيقظت بنشاط", 1),
    ("لا أشعر بأي تقدم", 0),
    ("يوم رائع مع أصدقائي", 1),
    ("فشلت مرة أخرى", 0),
    ("تلقيت مكالمة أسعدتني", 1),
    ("كل شيء ينهار من حولي", 0),
    ("استمتعت بالأجواء اليوم", 1),
    ("أشعر بالقلق المستمر", 0),
    ("كان اللقاء دافئًا ومليئًا بالحب", 1),
    ("لا أتحمل الضغط أكثر", 0),
    ("نجح مشروعي أخيرًا", 1),
    ("فقدت عملي اليوم", 0),
    ("قضيت وقتًا ممتعًا في الحديقة", 1),
    ("أنا خائف مما سيأتي", 0),
    ("تلقيت دعمًا كبيرًا من أصدقائي", 1),
    ("اليأس يسيطر علي", 0),
    ("رحلتي كانت مليئة بالفرح", 1),
    ("لا شيء يسعدني مؤخرًا", 0),
    ("أحببت الفيلم كثيرًا", 1),
    ("كلماتهم جرحتني", 0),
    ("تذوقت طعامًا رائعًا", 1),
    ("لا أرى فائدة من المحاولة", 0),
    ("ضحكنا كثيرًا اليوم", 1),
    ("حلمي تبخر", 0),
    ("لحظة اللقاء كانت ساحرة", 1),
    ("خسرت أقرب الناس إلي", 0),
    ("المشي في الطبيعة يريح أعصابي", 1),
    ("لم يصدقني أحد", 0),
    ("ابتسامة طفل جعلت يومي أفضل", 1),
    ("كل شيء أصبح صعبًا", 0),
    ("اليوم احتفلت بنجاحي", 1),
    ("انهار كل شيء في لحظة", 0),
    ("أمضيت وقتًا ممتعًا مع العائلة", 1),
    ("فقدت الأمل تمامًا", 0),
    ("قضيت يومًا رائعًا في الريف", 1),
    ("الناس لا يفهمونني", 0),
    ("استمتعت بالموسيقى والهدوء", 1),
    ("لا أشعر بالسعادة أبدًا", 0),
    ("الأصدقاء جلبوا لي السعادة", 1),
    ("تعبت من المحاولة", 0),
    ("كل لحظة كانت رائعة", 1),
    ("كل شيء فشل", 0),
    ("النجاح كان ثمرة جهدي", 1),
    ("لا أملك شيئًا أفرح به", 0)
]

# تجربة النموذج ومقارنة النتيجة
correct = 0

for i, (text, true_label) in enumerate(samples):
    result = classifier(text)[0]

    predicted_label = 1 if result["label"] == ("LABEL_1") else 0
    is_correct = predicted_label == true_label
    correct += is_correct

    print(f"{i+1}. \"{text}\"")
    print(f"   🔍 Model → {predicted_label} | 🎯 True → {true_label} | {'✔️ صح' if is_correct else '❌ غلط'}\n")

# حساب الدقة
accuracy = correct / len(samples)
print(f"✅ Accuracy: {accuracy * 100:.2f}%")